In [3]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch.nn import BCEWithLogitsLoss
import numpy as np
import networkx as nx

# Cargar datos
df = pd.read_csv('Reviews.csv')
df = df.head(500)  # Reducir tamaño del dataset para facilitar la demostración

# Normalizar las calificaciones
df['weight'] = df['Score'] / df['Score'].max()

# Mapeo de usuarios y productos a índices únicos
user_ids = df['UserId'].unique()
product_ids = df['ProductId'].unique()
user_id_map = {uid: idx for idx, uid in enumerate(user_ids)}
product_id_map = {pid: idx + len(user_ids) for idx, pid in enumerate(product_ids)}

# Crear grafo
G = nx.DiGraph()
for _, row in df.iterrows():
    user_node = user_id_map[row['UserId']]
    product_node = product_id_map[row['ProductId']]
    G.add_edge(user_node, product_node, weight=row['weight'])

# Preparar datos para PyTorch Geometric
edge_index = torch.tensor(list(G.edges())).t().contiguous()
num_nodes = len(user_id_map) + len(product_id_map)
x = torch.randn((num_nodes, 745))  # características aleatorias para nodos

data = Data(x=x, edge_index=edge_index)

class GATNetwork(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_heads):
        super(GATNetwork, self).__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=num_heads, dropout=0.6)
        self.conv2 = GATv2Conv(hidden_channels * num_heads, hidden_channels, heads=num_heads, concat=True, dropout=0.6)
        self.pred_layer = torch.nn.Linear(hidden_channels * num_heads * 2, 1)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        edge_embeddings = torch.cat((x[edge_index[0]], x[edge_index[1]]), dim=1)
        return self.pred_layer(edge_embeddings).squeeze()

model = GATNetwork(745, 8, 4)  # Configurar según las características de entrada
optimizer = Adam(model.parameters(), lr=0.005)
criterion = BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
data.to(device)

# Generar etiquetas correctamente alineadas con las aristas
labels = torch.randint(0, 2, (data.edge_index.size(1),), dtype=torch.float).to(device)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    predictions = torch.sigmoid(out)
    loss = criterion(predictions, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        predictions = torch.sigmoid(out) > 0.5
        correct = (predictions == labels).sum().item()
        total = labels.size(0)
        accuracy = correct / total
        return accuracy

# Bucle de entrenamiento
for epoch in range(100):
    loss = train()
    if epoch % 10 == 0:
        accuracy = evaluate()
        print(f'Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}')

Epoch 1: Loss = 0.7390, Accuracy = 0.5254
Epoch 11: Loss = 0.6879, Accuracy = 0.5051
Epoch 21: Loss = 0.6936, Accuracy = 0.4909
Epoch 31: Loss = 0.6921, Accuracy = 0.4970
Epoch 41: Loss = 0.6904, Accuracy = 0.5254
Epoch 51: Loss = 0.6805, Accuracy = 0.5558
Epoch 61: Loss = 0.6764, Accuracy = 0.6004
Epoch 71: Loss = 0.6709, Accuracy = 0.6389
Epoch 81: Loss = 0.6710, Accuracy = 0.6775
Epoch 91: Loss = 0.6688, Accuracy = 0.6937
